In [1]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
import pickle as pkl
from datetime import datetime
from elasticsearch import Elasticsearch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from elasticsearch import Elasticsearch

In [3]:
import jieba.posseg as psg
import pickle

In [10]:
def tongjiau(li):
    authors = []
    for i in li:
        try:
            authors += i
        except:
            continue
    return list(set(authors))

def savepickle(name, data):
    output = open('%s.pkl' % (name), 'wb')
    pkl.dump(data, output)
    output.close()

def loadpickle(name):
    pkl_file = open('%s.pkl' % (name), 'rb')
    data1 = pkl.load(pkl_file)
    return data1

def get_data(key, from_date, to_date, volume):
    es4Paper = esPaperRetrieval(host='10.8.128.205',port=49200)
    data= es4Paper.Retrieval(titleQuery=key,kwordQuery=key,summaryQuery=key,pubQuery='',
              fromDate=from_date,toDate=to_date,volume=volume)
    data = pd.DataFrame(data)[["P_Author", "P_Publication", "P_Summary_seg", "P_Keyword", "P_year"]]
    data.columns = ["authors", "Jounal", "seg_abstract", "keywords", "year"]
    data.year = [i[0:4] for i in data.year]

    group_df = data.groupby(by=['year'])
    # 统计每年的论文数量
    years = list(set(data['year']))
    # years.remove(None)
    years = sorted([int(i) for i in years])
    year_num = []
    for y in years:
        try:
            year_num.append(len(group_df.get_group(str(y))))
        except:
            year_num.append(0)

    # 根据年份将文献信息整合并保存

    years_df = pd.DataFrame()
    years_df['year'] = years
    seg = []
    key = []
    jou = []
    authors = []
    for i in years:
        seg.append(' '.join(list(group_df.get_group(str(i))['seg_abstract'])))
        k = list(group_df.get_group(str(i))['keywords'])
        if None in k:
            while None in k:
                k.remove(None)
            key.append(' '.join(k))
        else:
            key.append(' '.join(k))
        jou.append(' '.join(list(group_df.get_group(str(i))['Jounal'])))
        authors.append(' '.join(tongjiau(list(group_df.get_group(str(i))['authors']))).split())
    years_df['seg_abstract'] = seg
    years_df['keywords'] = key
    years_df['authors'] = authors
    years_df['Jounal'] = jou
    years_df['paper_num'] = year_num
    savepickle('year_df', years_df)
    return years_df

def print_top_words(model, feature_names, n_top_words):
    top_n = []
    for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
        top_n.append([feature_names[i]
                      for i in topic.argsort()[:-n_top_words - 1:-1]])

    return top_n

def build_tfidf(max_feature, data):
    #     vectorizer=CountVectorizer()
    tfidf = TfidfVectorizer(max_features=max_feature)
    f_tfidf = tfidf.fit_transform(data)
    word = tfidf.get_feature_names()
    f_tfidf = f_tfidf.toarray()
    return f_tfidf, word

import re
def read_s_words(stop_file):
    with open(stop_file, 'r', encoding='utf8') as f:
        stop_words = set()
        for line in f:
            stop_words.add(line.strip())
    return stop_words
def clean_words(strs):
    stop_words = read_s_words('../data/paper_data/stopwordPaper.txt')
    words = strs.split(' ')
    return ' '.join([word for word in words if word not in stop_words and not bool(re.match('.*?\d.*?', word))])


# 用tfidf计算每年的主题
# 用tfidf计算每年的主题
def tfidf_lda(n_topic, n_top_words, max_feature, data):
    tfidf = TfidfVectorizer(max_features=max_feature)
    f_tfidf = tfidf.fit_transform(data)
    f_tfidf = f_tfidf.toarray()
    # print(f_tfidf)
    # 主题的个数
    n_topics = n_topic
    # 主题词个数
    n_top_words = n_top_words
    n_samples = f_tfidf.shape[0]

    n_features = f_tfidf.shape[1]
    lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)

    lda.fit(f_tfidf)
    tf_feature_names = tfidf.get_feature_names()

    top_30 = print_top_words(lda, tf_feature_names, n_top_words)

    top_30_all = print_top_words(lda, tf_feature_names, n_top_words)
    top_30_find_weight=[]
    for h in range(n_topics):
        top_30 = top_30_all[h]
        top_30_weight = []
        for i in range(len(f_tfidf)):
            for j in range(len(tf_feature_names)):
                if tf_feature_names[j] in top_30:
                    top_30_weight.append(f_tfidf[i][j]+0.1)
        top_30_find_weight.append(top_30_weight)
    return top_30_all, top_30_find_weight

def zhutifenxi( key, n_topic=3, n_top_words=30, year_from=2010, year_to=2018, volume=500):
    years_df = get_data(key, str(year_from-1)+"-12-31", str(year_to)+"-12-31", volume)
    years_df['seg_abstract'] = years_df['seg_abstract'].map(clean_words)
    max_feature = 2000
    result = []
    value=[]
    result += tfidf_lda(n_topic, n_top_words, max_feature,years_df.loc[years_df.year.isin(range(year_from, year_to + 1))]['seg_abstract'])[0]
    value += tfidf_lda(n_topic, n_top_words, max_feature,years_df.loc[years_df.year.isin(range(year_from, year_to + 1))]['seg_abstract'])[1]
    key = [[("name", "value")] * n_top_words for i in range(n_topic)]
    return [[dict(zip(t[0],t[1])) for t in k]  for k in [list(zip(j[0], j[1])) for j in list(zip(key, [list(zip(i[0], i[1])) for i in list(zip(result, value))]))]]
def merge_list(x):
    y = []
    for line in x:
        line = line.split(' ')
        y = y +  line
    return y

def newtfidf_lda(n_topic, n_top_words, max_feature, data):
    tfidf = TfidfVectorizer(max_features=max_feature)
    f_tfidf = tfidf.fit_transform(data)
    f_tfidf = f_tfidf.toarray()
    # print(f_tfidf)
    # 主题的个数
    n_topics = n_topic
    # 主题词个数
    n_top_words = n_top_words
    n_samples = f_tfidf.shape[0]

    n_features = f_tfidf.shape[1]
    lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)

    lda.fit(f_tfidf)
    tf_feature_names = tfidf.get_feature_names()
    top_30 = print_top_words(lda, tf_feature_names, n_top_words)
    top_30 = top_30[0]
    top_30_weight = []
    print(top_30)
    word_valus = dict(zip(tf_feature_names, f_tfidf[0]))
    for word in top_30:
        top_30_weight.append(word_valus[word]+0.01)
    print(top_30_weight)
    return top_30, top_30_weight
def youxujulei(key, n_class=4, n_top_topic=30):
    years_df = get_data(key, "2010-12-31", "2019-12-31", 1000)
    # 设置特征维度
    years_df['seg_abstract'] = years_df['seg_abstract'].map(clean_words)
    max_feature = 2000
    # 根据年份建立tfidf表
    year_tfidf,tf_feature_names  = build_tfidf(max_feature, years_df['seg_abstract'])


    # 有序聚类

    # 聚类数量
    lst = get_one_essay_trace(year_tfidf[0], n_class)
    split_result = observe_divided(lst[0], years_df['year'])

    split_df = pd.DataFrame()
    abstract_agg = []
    abstracts = []
    for i in split_result:
        tmp = []
        for y in i:
            tmp.append(list(years_df.loc[years_df.year == y]['seg_abstract'])[0])
        abstract_agg.append(' '.join(tmp))
        abstracts.append(tmp)
    split_df['class'] = split_result
    split_df['merge_abstract'] = abstract_agg
    split_df['abstracts'] = abstracts

    # 根据tf计算LDA主题词
    # 算法 类名/第i类
    result_str = []
    keys = ["pub_id", "title", "publicationDate", "firstApplicant"]

    result1 = []
    value1 = []
    year = []
    for i in range(n_class):
        t = 1
        n_topic=1
        result, value = newtfidf_lda(n_topic, n_top_topic, max_feature, split_df.loc[i, 'seg_abstract'])
        result1.append(result)
        value1.append(value)
        year.append(str(split_df['class'][i]))
    key = [[("name", "value")] * n_top_topic for i in range(n_class)]
    # return result_str
    return [[dict(zip(t[0], t[1])) for t in k] for k in [list(zip(j[0], j[1])) for j in list(zip(key, [list(zip(i[0], i[1])) for i in list(zip(result1, value1))]))]],year


In [5]:
def distance(a, b):
    #  计算两个vector的距离，逐项差的平方和，不开根号
    if len(a) != len(b):
        print('wrong')
        return 0
    else:
        temp = a - b
        temp = temp ** 2
        res = sum(temp)
        return res
def get_dist_mat(feat):
    #  输入为特征矩阵，返回一个相似度矩阵（行数乘行数）
    print(type(feat))
    length=feat.shape[0]#样本数量
    wide = feat.shape[1]#特征向量长度
    res = np.zeros(shape=(length, length))
    ave = np.zeros(shape=(length, length, wide))

    for i in range(0, length):
        for j in range(i+1, length):
            ave[i, j] = np.mean(feat[i: j+1], axis=0)
            # print('i = %d, j = %d' % (i, j))

    for i in range(0, length):
        for j in range(i+1, length):
            ave_one = ave[i, j]
            dist_lst = [distance(ave_one, one) for one in feat[i: j+1]]
            res[i, j] = sum(dist_lst)
            # print('i = %d, j = %d' % (i, j))
    return res

def get_class_devide(dist_mat, class_num):
    #  利用递推公式计算分成class_num类别的分割方法
    length = len(dist_mat)
    divided_point = np.zeros(shape=(length, class_num+1))  # 存切割点
    diveded_dist = np.zeros(shape=(length, class_num+1))  # 存切割最小距离

    for i in range(1, length):  # 分成两类
        dist_lst = [dist_mat[0, k] + dist_mat[k+1, i] for k in range(0, i)]
        divided_point[i, 2] = np.argmin(dist_lst)
        diveded_dist[i, 2] = np.min(dist_lst)

    for classes in range(3, class_num+1):   # 分成多类
        for i in range(classes-1, length):
            dist_lst = [diveded_dist[k, classes-1] + dist_mat[k+1, i] for k in range(classes-2, i)]  # 从n-1类到n类
            divided_point[i, classes] = np.argmin(dist_lst) + classes - 2
            diveded_dist[i, classes] = np.min(dist_lst)
    return diveded_dist, divided_point

def get_trace(trace_mat, class_num):
    #  获得切割方法list，
    #  trace_mat就是get_class_devide函数返回的divided_point，分割点记录矩阵
    lst = []
    length = len(trace_mat)  #length=38
    pre = length - 1   #pre=37
    for i in range(class_num, 1, -1):
        temp = int(trace_mat[pre, i])
        pre = temp
        lst.append(temp)
    lst.reverse()
    return lst
def get_one_essay_trace(feature,phrase_num):
    #  整合调用 有序聚类方法
    #  调用就可以获得一个文件的分割结果并存入本地

    time_start = time.time()
    dist_mat = get_dist_mat(feature)
    phrase_num = phrase_num
    min_dist_mat, min_trace_mat = get_class_devide(dist_mat, phrase_num)
    # print(min_trace_mat)

    lst = get_trace(min_trace_mat, phrase_num)
    lst.append(len(feature))
    print('划分结果为：',end='')
    print(lst)
    time_end = time.time()
    print('cost time', time_end - time_start)
    return lst,min_dist_mat,min_trace_mat
def observe_divided(lst, sents):
    # 展示分割结果
    split_result=[]
    start = 0
    for end in lst:
        cla_lst=[]
        if end==len(sents):
            end=end-1
        for i in range(start, end+1):
            cla_lst.append(sents[i])
        start = end + 1
        split_result.append(cla_lst)
    return split_result

In [6]:
class esPaperRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esPaperRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esPaperRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'paper-detail-index'

    def do_search(self, titleQuery, kwordQuery, summaryQuery ,pubQuery,fromDate, toDate, volume):
        '''
        do_search方法执行具体检索过程
        titleQuery 为对应标题检索词
        kwordQuery 为对应关键字检索词
        summaryQuery 为对应摘要检索词
        上述三个词为or模式 可以出现 可以不出现
        pubQuery 为对应文摘关键词 这个关键词必定匹配
        fromDate 为检索字段起始日期 toDate为终结日期 日期格式 yyyy-mm-dd
        volume为每次检索返回的数目
        '''
        queryBody = {
          "query": {
            "bool": {
              "should": [
                {
                  "term": {
                    "P_Title": titleQuery
                  }
                },
                {
                  "term": {
                    "P_Keyword": kwordQuery
                  }
                },
                {
                  "term": {
                    "P_Summary": summaryQuery
                  }
                },
                  {
                  "term": {
                    "P_Publication.keyword": {"value":pubQuery}
                  }
                  }
              ],
              "filter": [
                {
                  "range": {
                    "P_year": {
                      "gt": fromDate,
                      "lt": toDate
                    }
                  }
                },
              ]
            }
          },
            "from": 0,
            "size": volume,
            "sort": [],
            "aggs": {}
        }
#         print(queryBody)
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'P_ID, P_Title, P_Author, P_Publication, P_Organ, P_year, P_Keyword,     P_Summary, P_Keyword_seg, P_Title_seg,    P_Summary_seg, P_URL, P_Fields, P_Fields_two,P_References, P_Pagecount, P_Page, P_Language,    P_Download_num, P_Citation_num,P_Vector,P_Volume, P_Issue,P_Issn,P_Isbn, P_Doi,    P_Red1, P_Red2, P_Red3, P_Red4, P_Red5'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs
    def Retrieval(self, titleQuery, kwordQuery, summaryQuery ,pubQuery,fromDate, toDate, volume):
        result = self.do_search(titleQuery, kwordQuery, summaryQuery ,pubQuery,fromDate, toDate, volume)
        docs = self.format_search(result)
        return docs



# 有序聚类

In [7]:
keys_map = {"pub_id":'algorithmName', "title":'className', "publicationDate":'topicName', "firstApplicant":'keywords'}

result = youxujulei('中医')

result = [{keys_map[k]:i[k] for k in i} for i in result]

<class 'numpy.ndarray'>


IndexError: tuple index out of range

# 主题聚类

In [11]:
result = zhutifenxi('中医',n_topic=7)

result = [{'topicID':num,'topicWords':i} for num,i in enumerate(result)]

In [12]:
result

[{'topicID': 0,
  'topicWords': [{'name': '患者', 'value': 0.10833994910896118},
   {'name': '认知损害', 'value': 0.11250992366344176},
   {'name': '表达', 'value': 0.1},
   {'name': '模型', 'value': 0.1},
   {'name': '轻至', 'value': 0.1},
   {'name': '治疗', 'value': 0.1828019029241683},
   {'name': '起始', 'value': 0.10337595378660806},
   {'name': '康复治疗', 'value': 0.11350381514643224},
   {'name': '中线', 'value': 0.1},
   {'name': '左旋多巴', 'value': 0.1},
   {'name': '入院前', 'value': 0.12115633588735031},
   {'name': '持续气道正压通气', 'value': 0.1},
   {'name': '训练', 'value': 0.15289083971837575},
   {'name': '中位', 'value': 0.11012786135982418},
   {'name': '倾向', 'value': 0.6630529398843444},
   {'name': '卡片', 'value': 0.12115633588735031},
   {'name': '小脑后下动脉', 'value': 0.1},
   {'name': 'bcl', 'value': 0.13036069773886172},
   {'name': 'sanger测序', 'value': 0.12025572271964834},
   {'name': '海人酸', 'value': 0.2297229812478637},
   {'name': '顺利完成', 'value': 0.1041699745544806},
   {'name': '临床', 'value': 0.1